In [1]:
using Printf
using Random
using LinearAlgebra
using Eirene
using MAT
using Statistics
using Plots
using Glob

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-3733468780211953841\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-15233383457869095681\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-3733468780211953841\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/sppatankar/.julia/packages/Plots/oZheM/src/backends/hdf5.jl", 162, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010ac1511f, Ptr{Nothing} @0x000000010acaeeb3, Ptr{Nothing} @0x000000010acaffab, Ptr{Nothing} @0x000000010acae8df, Ptr{Nothing} @0x000000010acaebbc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010acc81ce, Ptr{Nothing} @0x000000010acc71b1, Ptr{Nothing} @0x000000010acc79d1, Ptr{Nothing} @0x000000010acc80d6, Ptr{Nothing} @0x000000010aca0b6d, Ptr{Nothing} @0x000000010acc927c, Ptr{Nothing} @0x0000000115df1d4f, Ptr{Nothing} @0x000000013aac14ae, Ptr{Nothing} @0x000000010acaffcf, Ptr{Nothing} @0x000000010acae8df, Ptr{Nothing} @0x000000010acaebbc, Base.InterpreterIP in top-level CodeInfo for Plots at statement 10, Ptr{Nothing} @0x000000010acc81ce, Ptr{Nothing} @0x000000010acc9074, Ptr{Nothing} @0x000

In [2]:
function constant_probability(n, p)
    G = zeros(n, n)
    for i = 1:n
        for j = 1:i-1
            r = rand(1)[1]
            if r < p
                G[i, j] = 1
                G[j, i] = 1
            end
        end
    end
    node_order = 1:n
    return G, node_order
end  

function make_weighted_from_order(G, node_order)
    # adapted from code by ASB
    # original at https://github.com/BassettLab/Reorderability_scripts
    reordered_G = G[node_order, node_order] # often unnecessary
    n = length(node_order) # number of nodes
    val_mat = ones(n, n)
    for col = 1:n
        val_mat[1:col, col] = val_mat[1:col, col] * col
        val_mat[col, 1:col] = val_mat[col, 1:col] * col
    end
    weighted_G = reordered_G .* val_mat
    # replace 0 weighted edges with the largest edge weight possible
    # this is equivalent to assigning these edges the worst rank possible
    replace!(weighted_G, 0 => 2 * n)  
    # weighted_G[findall(A -> A .== 0, weighted_G)] .= 2 * n would work too
    weighted_G[diagind(weighted_G)] .= 0 # set diagonal to 0
    return weighted_G # edges here are ranked by order of appearance
end

function bettiCurveFromBarcode(barcode_array,nNodes,nmats,maxDim)
    nNodes = Int(nNodes)
    nmats = Int(nmats)
    maxDim = Int(maxDim)
    bettiBar = zeros(nmats,maxDim)
    bettiCurve = zeros(nmats,nNodes+1,maxDim)
    birthCurve = zeros(nmats,nNodes,maxDim)
    deathCurve = zeros(nmats,nNodes,maxDim)
    for dimn in collect(1:maxDim)
        dimn = Int(dimn)
        for matn in collect(1:nmats)
            matn = Int(matn)
            bb = 0
            currentCurve = barcode_array[matn,:]
            currentCurveDim = currentCurve[dimn]
            for barn in collect(1:size(currentCurveDim,1))
                # Add to birth curve
                birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] = birthCurve[matn,Int(currentCurveDim[barn,1]),dimn] .+1
                if currentCurveDim[barn,2]>nNodes
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(nNodes+1),dimn] .+1
                    bb = bb+(nNodes+1-currentCurveDim[barn,1])
                else
                    bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn] = bettiCurve[matn,Int(currentCurveDim[barn,1]):Int(currentCurveDim[barn,2]),dimn].+1
                    deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] = deathCurve[matn,Int(currentCurveDim[barn,2]),dimn] .+1
                    bb = bb+(currentCurveDim[barn,2] - currentCurveDim[barn,1])
                end
            end
            bettiBar[matn,dimn] = deepcopy(bb)
        end
    end
    return bettiCurve, birthCurve, deathCurve, bettiBar
end

function plotBarcode(allPIs,nNodes,graphN,maxDim,fontSize)
    nNodes = Int(nNodes)
    graphn = Int(graphN)
    maxDim = Int(maxDim)
    counter1 = 0
    pbar = plot(1:6,zeros(6),c=:black)
    colors = [:blue :green :red]
    for dim in collect(1:maxDim)
        barn = barcode_array[graphN, dim]
        barn = barn[sortperm(barn[:,1]),:]
        nbars = size(barn)[1]
        for cntr1 in collect(1:nbars)
            birth = barn[cntr1,1]
            death = barn[cntr1,2]
            plot!([birth, death],[cntr1+counter1, cntr1+counter1],c=colors[dim], legend = false,
                            xlim = (0,nNodes), ytickfont = font(fontSize), xtickfont = font(fontSize))
        end
        display(pbar)
        counter1 = counter1+nbars
    end
    return pbar
end

plotBarcode (generic function with 1 method)

In [3]:
pwd()

"/Volumes/My Passport/Curiosity/v3"

In [4]:
data_in_path = "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene"
data_out_path = "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/reordered_KNOT_preprocessed_Eirene"
all_subjs = glob("*.mat", data_in_path)

149-element Array{String,1}:
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_101.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_104.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_105.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_106.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_107.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_108.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_109.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_112.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_114.mat"
 "/Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Ei

In [5]:
iters = 5
for i = 1:length(all_subjs)
    @printf("Subject: %s\n", all_subjs[i])
    all_vars = matread(all_subjs[i])
    nodes = all_vars["nodes"]
    n = length(nodes)
    adj = all_vars["adj"]
    weighted_G = make_weighted_from_order(adj, 1:n)
    subj_ID = all_vars["subj"]
    reordered_nodes = zeros(1, n, iters)
    reordered_adjs = zeros(n, n, iters)
    reordered_weighted_Gs = zeros(n, n, iters)
    for j = 1:iters
        new_node_order = randperm(n);
        reordered_nodes[1, :, j] = new_node_order
        reordered_adj = adj[new_node_order, new_node_order]
        reordered_adjs[:, :, j] = reordered_adj
        reordered_weighted_Gs[:, :, j] = make_weighted_from_order(reordered_adj, new_node_order)
    end
    save_string = string(data_out_path, "/", "subj_", subj_ID, ".mat")
    matwrite(save_string, 
        Dict("subj" => subj_ID,
            "adj" => adj,
            "weighted_G" => weighted_G,
            "reordered_nodes" => reordered_nodes,
            "reordered_adjs" => reordered_adjs,
            "reordered_weighted_Gs" => reordered_weighted_Gs))
end

Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_101.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_104.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_105.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_106.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_107.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_108.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_109.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_112.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_114.mat
Subject: /Users/sppatankar/Developer/My Passport/Curios

Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_231.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_232.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_234.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_235.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_236.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_238.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_239.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_240.mat
Subject: /Users/sppatankar/Developer/My Passport/Curiosity/v3/Data/KNOT_preprocessed_Eirene/subj_242.mat
Subject: /Users/sppatankar/Developer/My Passport/Curios